## Constraints and Triggers Activity

Constraints and triggers are tools to impose restrictions on allowable data within a database, beyond the requirements imposed by table definition types.

**Constraints**, also known as _integrity constraints_, are used to constrain allowable database states.  They prevent disallowed values from being entered into the database. 
* non-null constraints
  * `create Table MyTable(myValue dataType NOT NULL);`
* key or uniqueness constraints 
  * `create Table MyTable(myId int PRIMARY KEY);`
  * `create Table MyTable(myValue1 dataType, myValue2 dataType, UNIQUE(myValue1,myValue2));`
* attribute restrictions
  * `create Table MyTable(myValue dataType check(myValue > 0))`
* referential integrity  (a.k.a. foreign keys)
  * `create Table MyTable(otherId int, foreign key(otherId) references OtherTable(otherColumn))`

**Triggers** are procedures that get run when specified events in a database view or table occur.  They are useful for implementing monitoring logic at the database level.
* delete/update/insert
* before/after/instead of
* when(condition)
* row-level/statement level


## Activity 1 - Constraints

Write CREATE TABLE declarations with the necessary constraints for the following 4 tables and their specifications:

* `Student(sID, name, parentEmail, gpa)`
  * `sID (should be unique)`
  * `name (should exist)`
  * `parentEmail(should exist)`
  * `gpa (real value between 0 and 4 inclusive)`
* `Class(cID, name, units)`
  * `cID (should be unique)`
  * `name (should exist)`
  * `units (must be between 1 and 5 inclusive)`
* `ClassGrade(sID, cID, grade)`
  * `sID (should reference a student)`
  * `cID (should reference a class)`
  * `grade (integer between 0 and 4 inclusive, for F,D,C,B,A)`
  * `student can only get 1 grade for each class`
* `ParentNotification(parentEmail, text)`
  * `parentEmail (should exist)`
  * `text (the message body, should exist)`

In [1]:
%load_ext sql
%sql sqlite://

/Users/jennifer/Library/Python/2.7/lib/python/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/jennifer/Library/Python/2.7/lib/python/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


'Connected: None@None'

Write your table definitions here:

In [54]:
%%sql
DROP TABLE IF EXISTS Student;
CREATE TABLE Student(
    sID INT PRIMARY KEY,
    name TEXT NOT NULL, 
    parentEmail TEXT NOT NULL,
    gpa DOUBLE CHECK (gpa >= 0 AND gpa <= 4)
);

DROP TABLE IF EXISTS Class;
CREATE TABLE Class(
    cID INT PRIMARY KEY,
    name TEXT NOT NULL,
    units INT CHECK (units >= 1 AND units <= 5)
);

DROP TABLE IF EXISTS ClassGrade;
CREATE TABLE ClassGrade(
    sID INT,
    cID INT,
    grade INT, CHECK (grade >= 0 AND grade <= 4),
    FOREIGN KEY(sID) REFERENCES Student(sID),
    FOREIGN KEY(cID) REFERENCES Class(cID),
    PRIMARY KEY (sID, cID)
);

DROP TABLE IF EXISTS ParentNotification;
CREATE TABLE ParentNotification(
    parentEmail TEXT NOT NULL,
    text TEXT NOT NULL
);



Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

## Activity 2 - Triggers Introduction

Triggers are used to execute sql commands upon changes to the specified tables.  Trigger support in SQLite can be found [here](https://www.sqlite.org/lang_createtrigger.html).

The following is an example of a trigger in SQLite.

In [55]:
%%sql
drop table if exists Employee;
drop table if exists Department;
drop trigger if exists update_employee_count;
create table Employee(eID int, name text, dID);
create table Department(dID int, name text, employee_count int);

Done.
Done.
Done.
Done.
Done.


[]

In [56]:
%%sql
create trigger update_employee_count
after insert on Employee
for each row
begin
  update Department set employee_count = employee_count + 1 where
  dID = new.dID;
end;

Done.


[]

Note that there is a difference between OLD values and NEW values in triggers that execute on statements that change values in a table.  Both the WHEN clause and the trigger actions may access elements of the row being inserted, deleted or updated using references of the form "NEW.column-name" and "OLD.column-name", where column-name is the name of a column from the table that the trigger is associated with.  Triggers on INSERT statements (like that above) can only access the NEW values (since OLD values don't exist!) and triggers on DELETE statements can only access OLD values.

Let's continue by adding data to the tables.

In [57]:
%%sql
insert into Department values(1,'HR',0);
insert into Department values(2,'Engineering',0);

1 rows affected.
1 rows affected.


[]

At this point, there are no empoloyees in the Employee table.  As you can see below, each department has 0 employees.

In [58]:
%%sql
select name, employee_count
from department;

Done.


name,employee_count
HR,0
Engineering,0


When we insert several employees into the Employee table, the trigger should fire and update values in the Department table.

In [59]:
%%sql
insert into Employee values
(1,'Todd',1),(2,'Jimmy',1),(3,'Billy',2);

3 rows affected.


[]

Now when we view the employee table, we see that the employee count has been updated by the trigger.

In [60]:
%%sql
select name, employee_count
from department;

Done.


name,employee_count
HR,2
Engineering,1


Now, it's your turn!  Write a SQLite trigger on the ClassGrade table you defined earlier.  On each insertion into the ClassGrade table, the trigger should update the GPA of the corresponding student.
* `gpa = sum(units*grade)/sum(units)`

First, let's load data into the tables:

In [61]:
%%sql
insert into Student values(1,'Timmy','timmysmom@gmail.com', 0.0);
insert into Student values(2,'Billy','billysmom@gmail.com',0.0);
insert into Class values(1, 'CS145',4);
insert into Class values(2,'CS229',3);

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [62]:
%%sql
select * from student;

Done.


sID,name,parentEmail,gpa
1,Timmy,timmysmom@gmail.com,0.0
2,Billy,billysmom@gmail.com,0.0


Now, write your trigger here:

In [63]:
%%sql
DROP TRIGGER IF EXISTS update_student_gpa;
CREATE TRIGGER update_student_gpa
    AFTER INSERT ON ClassGrade
    FOR EACH ROW 
    BEGIN 
        UPDATE Student
        Set gpa = (
            SELECT SUM(units * grade) * 1.0/ SUM(units)
            FROM Class, ClassGrade
            WHERE Class.cID = ClassGrade.cID AND ClassGrade.sID = NEW.sID
        )
        WHERE Student.sID = NEW.sID;
    END

Done.
Done.


[]

Now, write a second trigger here that inserts a row in ParentNotification with the parent's email and a message.  The trigger should execute whenever a Student record is updated with a new GPA and that GPA is < 2.0.

A trigger like this can have a format similar to the following in SQLite:
```
create trigger XYZ
    after update of myColumn on myTable
    for each row when (condition in myTable)
    begin
        insert/update/delete etc.
     end
```

Write your trigger here:

In [64]:
%%sql
DROP TRIGGER IF EXISTS inform_parent;
CREATE TRIGGER inform_parent
    AFTER UPDATE OF gpa ON Student
    FOR EACH ROW WHEN (NEW.gpa < 2.0)
    BEGIN
        INSERT INTO ParentNotification VALUES(NEW.parentEmail, "Low GPA");
    END 


Done.


[]

We can now test the triggers.  

Done.


[]

In [65]:
%%sql
insert into ClassGrade values(1,1,2);
insert into ClassGrade values(1,2,1);
insert into ClassGrade values(2,1,1);
select * from ParentNotification;


1 rows affected.
1 rows affected.
1 rows affected.
Done.


parentEmail,text
timmysmom@gmail.com,Low GPA
billysmom@gmail.com,Low GPA


In [66]:
%%sql 
SELECT * FROM Student

Done.


sID,name,parentEmail,gpa
1,Timmy,timmysmom@gmail.com,1.57142857143
2,Billy,billysmom@gmail.com,1.0


## Activity 3 - Advanced Triggers

Triggers can execute BEFORE, AFTER, or INSTEAD OF the sql statements that trigger them.  [SQLite notes](https://www.sqlite.org/lang_createtrigger.html) that programmers should be very wary when executing BEFORE or INSTEAD OF triggers.

> If a BEFORE UPDATE or BEFORE DELETE trigger modifies or deletes a row that was to have been updated or deleted, then the result of the subsequent update or delete operation is undefined. Furthermore, if a BEFORE trigger modifies or deletes a row, then it is undefined whether or not AFTER triggers that would have otherwise run on those rows will in fact run.

> The value of NEW.rowid is undefined in a BEFORE INSERT trigger in which the rowid is not explicitly set to an integer.

> Because of the behaviors described above, programmers are encouraged to prefer AFTER triggers over BEFORE triggers.

**Triggers are one of the unfortunate areas where SQL implementations differ greatly.  The correct semantics for a row-level “after” trigger, according to the SQL standard, is to activate the trigger after the entire triggering data modification statement completes, executing the trigger once for each modified row. PostgreSQL implements these semantics. SQLite instead implements semantics where the trigger is activated immediately after each row-level change, interleaving trigger execution with execution of the modification statement.**

Finally, SQLite supports the RAISE() function.  The function can be used to halt the execution of a trigger and the statement that caused it.  Here's an example that would prevent students from getting a grade in CS 245 until they've gotten a B or better in CS 145.



In [67]:
%%sql
drop trigger if exists enforce_cs245_prereqs;

insert into Class values (3,'CS245',3);
insert into Student values (3,'Johnny', 'johnnysmom@gmail.com', 0.0);
insert into ClassGrade values (3,1,4);

create trigger enforce_cs245_prereqs
before insert on ClassGrade
for each row
when exists (
        Select * 
        from Class c1
        where c1.cID = new.cID
        and c1.name = 'CS245'
        and new.sID not in (
            Select cg.sID
            from class c2, ClassGrade cg
            where c2.cID = cg.cID
            and c2.name = 'CS145'
            and cg.grade > 2)
    )
begin
  select raise(rollback, 'A student must pass CS 145 before taking CS 245');
end;

Done.
1 rows affected.
1 rows affected.
1 rows affected.
Done.


[]

With our trigger, student number 3, Johnny, should be able to take CS 245 since he got an A in CS 145.  

In [68]:
%%sql
insert into ClassGrade values (3,3,4.0);

1 rows affected.


[]

In [69]:
%sql select * from ClassGrade;

Done.


sID,cID,grade
1,1,2
1,2,1
2,1,1
3,1,4
3,3,4


As you can see, Johnny had no trouble getting a grade in the class.  Now, if we try to enter a grade for Student 1, it should fail due to our trigger.  It will present a rollback message if the trigger executes.

In [70]:
%%sql
insert into ClassGrade values (1,3,4.0);

IntegrityError: (sqlite3.IntegrityError) A student must pass CS 145 before taking CS 245 [SQL: u'insert into ClassGrade values (1,3,4.0);']

Now, it's your turn!  Write a trigger that prevents a student from getting a grade in any class when there are pending emails in the ParentNotification table for that student's parent.

In [ ]:
%%sql
DROP TRIGGER IF EXISTS no_parentnotification;

CREATE TRIGGER no_parentnotification
BEFORE INSERT ON ClassGrade
FOR EACH ROW
WHEN EXISTS (
    SELECT *
    FROM ParentNotification p, Student s
    WHERE p.parentEmail = s.parentEmail AND s.sID = NEW.sID
)
BEGIN
SELECT RAISE(ROLLBACK, 'A Student mu')
begin
  select raise(rollback, 'A student must pass CS 145 before taking CS 245');
end;

Assuming your trigger is correct, this statement should succeed.

In [ ]:
%%sql
insert into ClassGrade values (3,2,4);

And this one shoud fail.

In [ ]:
%%sql
insert into ClassGrade values(2,2,1);

## Further Information

At this point, you should have all the knowledge you need to do Project Part 2.  If you want to learn more about constraints and triggers, you can view the videos from Professor Widon below.  Please note that this information is **NOT** necessary for any exams or homeworks that you will have in this class.  You do not need it to do the constraints or triggers on the project.

* [Constraints and Triggers](https://www.youtube.com/watch?v=kaI9rQ6e7xQ&feature=youtu.be)
* [Types of Constraints](https://www.youtube.com/watch?v=EGeDJzdh--o&feature=youtu.be)
* [Referential Integrity](https://www.youtube.com/watch?v=zFeSglmrY78&feature=youtu.be)
* [Triggers](https://www.youtube.com/watch?v=s9iRD-zrxS0&feature=youtu.be)
* [Trigger demo 1](https://www.youtube.com/watch?v=i3xwZw5VRMs&feature=youtu.be)
* [Trigger demo 2](https://www.youtube.com/watch?v=GSYhvQVI_dE&feature=youtu.be)